<a href="https://colab.research.google.com/github/ewertonsilvag/mini_projeto_final_SoulCode/blob/main/Normalizacoes_Spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instalando o Pyspark

In [ ]:
pip install pyspark

Importando as bibliotecas

In [ ]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType, FloatType, DoubleType
from pyspark.sql.functions import *
from pyspark.sql.window import Window


Configurando o SparkSession

In [ ]:
spark = (SparkSession.builder\
        .master("local")\
        .appName("dataframe_withcolumn")\
        .config("spark.ui.port", "4050")\
        .getOrCreate())
spark

Montando o DataFrame com o StructType e chamando-o do Drive

In [ ]:
schema = StructType([ 
      StructField("id",IntegerType(),True),
      StructField("ano_nascimento",IntegerType(),True),
      StructField("educacao",StringType(),True),
      StructField("estado_civil",StringType(),True),
      StructField("renda",DoubleType(),True),
      StructField("qtd_crianca",IntegerType(),True),
      StructField("qtd_adolescente",IntegerType(),True),
      StructField("primeira_data_compra",DateType(),True),
      StructField("dias_desde_ult_compras",IntegerType(),True),
      StructField("qtd_vinhos",IntegerType(),True),
      StructField("qtd_frutas",IntegerType(),True),
      StructField("qtd_carne",IntegerType(),True),
      StructField("qtd_peixe",IntegerType(),True),
      StructField("guloseimas",IntegerType(),True),
      StructField("qtd_produtos_gold",IntegerType(),True),
      StructField("qtd_ofertas_compra",IntegerType(),True),
      StructField("qtd_compras_online",IntegerType(),True),
      StructField("qtd_compras_catalogo",IntegerType(),True),
      StructField("qtd_compras_na_loja",IntegerType(),True),
      StructField("qtd_visitas_online_mes",IntegerType(),True),
      StructField("oferta_aceita1",StringType(),True),
      StructField("oferta_aceita2",StringType(),True),
      StructField("oferta_aceita3",StringType(),True),
      StructField("oferta_aceita4",StringType(),True),
      StructField("oferta_aceita5",StringType(),True),
      StructField("reclamacao",StringType(),True)])

local= ("/content/drive/MyDrive/SoulCode/MiniProjeto/marketing_dataframe.csv")

df = spark.read.load(local, format="csv", header="true", sep=',', schema=schema)

Exibindo a tabela

In [ ]:
df.show()

+----+--------------+------------------+-------------+-------+-----------+---------------+--------------------+----------------------+----------+----------+---------+---------+----------+-----------------+------------------+------------------+--------------------+-------------------+----------------------+--------------+--------------+--------------+--------------+--------------+----------+
|  id|ano_nascimento|          educacao| estado_civil|  renda|qtd_crianca|qtd_adolescente|primeira_data_compra|dias_desde_ult_compras|qtd_vinhos|qtd_frutas|qtd_carne|qtd_peixe|guloseimas|qtd_produtos_gold|qtd_ofertas_compra|qtd_compras_online|qtd_compras_catalogo|qtd_compras_na_loja|qtd_visitas_online_mes|oferta_aceita1|oferta_aceita2|oferta_aceita3|oferta_aceita4|oferta_aceita5|reclamacao|
+----+--------------+------------------+-------------+-------+-----------+---------------+--------------------+----------------------+----------+----------+---------+---------+----------+-----------------+-------

Modificando o nome de duas colunas

In [ ]:
df=df.withColumnRenamed("id", "identificacao").withColumnRenamed("primeira_data_compra", "primeira_compra")
df.show()

+-------------+--------------+------------------+-------------+-------+-----------+---------------+---------------+----------------------+----------+----------+---------+---------+----------+-----------------+------------------+------------------+--------------------+-------------------+----------------------+--------------+--------------+--------------+--------------+--------------+----------+
|identificacao|ano_nascimento|          educacao| estado_civil|  renda|qtd_crianca|qtd_adolescente|primeira_compra|dias_desde_ult_compras|qtd_vinhos|qtd_frutas|qtd_carne|qtd_peixe|guloseimas|qtd_produtos_gold|qtd_ofertas_compra|qtd_compras_online|qtd_compras_catalogo|qtd_compras_na_loja|qtd_visitas_online_mes|oferta_aceita1|oferta_aceita2|oferta_aceita3|oferta_aceita4|oferta_aceita5|reclamacao|
+-------------+--------------+------------------+-------------+-------+-----------+---------------+---------------+----------------------+----------+----------+---------+---------+----------+-------------

Criando uma nova coluna com base na preferencia de proteína

In [ ]:
df1 = (df.withColumn("proteina_preferida", 
       F.when(F.col("qtd_carne") > F.col("qtd_peixe"), F.lit("prefere carne"))
        .otherwise("prefere peixe")))
df1.show()

+-------------+--------------+------------------+-------------+-------+-----------+---------------+---------------+----------------------+----------+----------+---------+---------+----------+-----------------+------------------+------------------+--------------------+-------------------+----------------------+--------------+--------------+--------------+--------------+--------------+----------+------------------+
|identificacao|ano_nascimento|          educacao| estado_civil|  renda|qtd_crianca|qtd_adolescente|primeira_compra|dias_desde_ult_compras|qtd_vinhos|qtd_frutas|qtd_carne|qtd_peixe|guloseimas|qtd_produtos_gold|qtd_ofertas_compra|qtd_compras_online|qtd_compras_catalogo|qtd_compras_na_loja|qtd_visitas_online_mes|oferta_aceita1|oferta_aceita2|oferta_aceita3|oferta_aceita4|oferta_aceita5|reclamacao|proteina_preferida|
+-------------+--------------+------------------+-------------+-------+-----------+---------------+---------------+----------------------+----------+----------+------

Criando uma nova coluna com base se o cliente já fez compra online

In [ ]:
df2 = (df1.withColumn("comprou_online", 
       F.when(F.col("qtd_compras_online") > 0, F.lit("já comprou"))
        .otherwise("não comprou")))
df2.show()

+-------------+--------------+------------------+-------------+-------+-----------+---------------+---------------+----------------------+----------+----------+---------+---------+----------+-----------------+------------------+------------------+--------------------+-------------------+----------------------+--------------+--------------+--------------+--------------+--------------+----------+------------------+--------------+
|identificacao|ano_nascimento|          educacao| estado_civil|  renda|qtd_crianca|qtd_adolescente|primeira_compra|dias_desde_ult_compras|qtd_vinhos|qtd_frutas|qtd_carne|qtd_peixe|guloseimas|qtd_produtos_gold|qtd_ofertas_compra|qtd_compras_online|qtd_compras_catalogo|qtd_compras_na_loja|qtd_visitas_online_mes|oferta_aceita1|oferta_aceita2|oferta_aceita3|oferta_aceita4|oferta_aceita5|reclamacao|proteina_preferida|comprou_online|
+-------------+--------------+------------------+-------------+-------+-----------+---------------+---------------+---------------------

Criando uma nova coluna para a quantidade total de filhos do cliente

In [ ]:
df2 = df2.withColumn("qtd_filhos", F.col("qtd_crianca") + F.col("qtd_adolescente"))
df2.show()

+-------------+--------------+------------------+-------------+-------+-----------+---------------+---------------+----------------------+----------+----------+---------+---------+----------+-----------------+------------------+------------------+--------------------+-------------------+----------------------+--------------+--------------+--------------+--------------+--------------+----------+------------------+--------------+----------+
|identificacao|ano_nascimento|          educacao| estado_civil|  renda|qtd_crianca|qtd_adolescente|primeira_compra|dias_desde_ult_compras|qtd_vinhos|qtd_frutas|qtd_carne|qtd_peixe|guloseimas|qtd_produtos_gold|qtd_ofertas_compra|qtd_compras_online|qtd_compras_catalogo|qtd_compras_na_loja|qtd_visitas_online_mes|oferta_aceita1|oferta_aceita2|oferta_aceita3|oferta_aceita4|oferta_aceita5|reclamacao|proteina_preferida|comprou_online|qtd_filhos|
+-------------+--------------+------------------+-------------+-------+-----------+---------------+---------------

Mostra qual tipo de cliente (agrupando pela escolaridade) o mercado mais atende

In [ ]:
df2.groupby(F.col("educacao")).count().show()


+------------------+-----+
|          educacao|count|
+------------------+-----+
| cursando_mestrado|  201|
|            mestre|  370|
|         doutorado|  485|
|         graduacao| 1127|
|ensino_fundamental|   54|
+------------------+-----+



Identificando os clientes por seu id, escolaridade e estado civil e a quantidade de vinhos que ele já comprou

In [ ]:
df2.select(F.col("identificacao"), F.col("educacao"), F.col("estado_civil"), F.col("qtd_vinhos")).filter(F.col("estado_civil") == "solteiro").show()

+-------------+-----------------+------------+----------+
|identificacao|         educacao|estado_civil|qtd_vinhos|
+-------------+-----------------+------------+----------+
|         5524|        graduacao|    solteiro|       635|
|         2174|        graduacao|    solteiro|        11|
|         2114|        doutorado|    solteiro|      1006|
|         2278|cursando_mestrado|    solteiro|         4|
|         7892|        graduacao|    solteiro|         6|
|         5255|        graduacao|    solteiro|         5|
|        10738|           mestre|    solteiro|        40|
|         7281|        doutorado|    solteiro|        81|
|         1371|        graduacao|    solteiro|       123|
|         7244|        graduacao|    solteiro|        48|
|         1331|        graduacao|    solteiro|        12|
|         7437|        graduacao|    solteiro|        13|
|         8557|        graduacao|    solteiro|        11|
|         6853|           mestre|    solteiro|       712|
|         1012

Filtrando os clientes que tem doutorado e são casados

In [ ]:
filtro1 = F.col("educacao") == "doutorado"
filtro2 = F.col("estado_civil") == "casado(a)"
df2.select(F.col("identificacao"), 
           F.col("educacao"),
           F.col("estado_civil")).filter(filtro1 & filtro2).show()

+-------------+---------+------------+
|identificacao| educacao|estado_civil|
+-------------+---------+------------+
|         5324|doutorado|   casado(a)|
|         6177|doutorado|   casado(a)|
|         1993|doutorado|   casado(a)|
|         4047|doutorado|   casado(a)|
|         1966|doutorado|   casado(a)|
|         4339|doutorado|   casado(a)|
|         6566|doutorado|   casado(a)|
|         8375|doutorado|   casado(a)|
|         8132|doutorado|   casado(a)|
|         8996|doutorado|   casado(a)|
|         2230|doutorado|   casado(a)|
|         3037|doutorado|   casado(a)|
|         9370|doutorado|   casado(a)|
|         9215|doutorado|   casado(a)|
|         8268|doutorado|   casado(a)|
|         5290|doutorado|   casado(a)|
|         9579|doutorado|   casado(a)|
|         9260|doutorado|   casado(a)|
|         8475|doutorado|   casado(a)|
|         8717|doutorado|   casado(a)|
+-------------+---------+------------+
only showing top 20 rows



Mostrando quando foi a última vez que os clientes (ordenados pela identificação) compraram no mercado.

In [ ]:
df2.select(F.col("identificacao"),
           F.col("dias_desde_ult_compras")).orderBy(F.col("identificacao").asc()).show(truncate=False)

+-------------+----------------------+
|identificacao|dias_desde_ult_compras|
+-------------+----------------------+
|0            |66                    |
|1            |0                     |
|9            |86                    |
|13           |57                    |
|17           |81                    |
|20           |91                    |
|22           |99                    |
|24           |96                    |
|25           |9                     |
|35           |35                    |
|48           |97                    |
|49           |39                    |
|55           |83                    |
|67           |6                     |
|73           |81                    |
|75           |32                    |
|78           |51                    |
|87           |16                    |
|89           |49                    |
|92           |12                    |
+-------------+----------------------+
only showing top 20 rows



Retorna o número da linha de acordo com a coluna que foi particionada



In [ ]:
w0 = Window.partitionBy(F.col("educacao")).orderBy("renda")
df2.withColumn("row_number", F.row_number().over(w0)).show()


+-------------+--------------+-----------------+-------------+-------+-----------+---------------+---------------+----------------------+----------+----------+---------+---------+----------+-----------------+------------------+------------------+--------------------+-------------------+----------------------+--------------+--------------+--------------+--------------+--------------+----------+------------------+--------------+----------+----------+
|identificacao|ano_nascimento|         educacao| estado_civil|  renda|qtd_crianca|qtd_adolescente|primeira_compra|dias_desde_ult_compras|qtd_vinhos|qtd_frutas|qtd_carne|qtd_peixe|guloseimas|qtd_produtos_gold|qtd_ofertas_compra|qtd_compras_online|qtd_compras_catalogo|qtd_compras_na_loja|qtd_visitas_online_mes|oferta_aceita1|oferta_aceita2|oferta_aceita3|oferta_aceita4|oferta_aceita5|reclamacao|proteina_preferida|comprou_online|qtd_filhos|row_number|
+-------------+--------------+-----------------+-------------+-------+-----------+------------

Rank da coluna particionada 

In [ ]:
df2.withColumn("rank", F.rank().over(w0)).show()


+-------------+--------------+-----------------+-------------+-------+-----------+---------------+---------------+----------------------+----------+----------+---------+---------+----------+-----------------+------------------+------------------+--------------------+-------------------+----------------------+--------------+--------------+--------------+--------------+--------------+----------+------------------+--------------+----------+----+
|identificacao|ano_nascimento|         educacao| estado_civil|  renda|qtd_crianca|qtd_adolescente|primeira_compra|dias_desde_ult_compras|qtd_vinhos|qtd_frutas|qtd_carne|qtd_peixe|guloseimas|qtd_produtos_gold|qtd_ofertas_compra|qtd_compras_online|qtd_compras_catalogo|qtd_compras_na_loja|qtd_visitas_online_mes|oferta_aceita1|oferta_aceita2|oferta_aceita3|oferta_aceita4|oferta_aceita5|reclamacao|proteina_preferida|comprou_online|qtd_filhos|rank|
+-------------+--------------+-----------------+-------------+-------+-----------+---------------+--------

In [ ]:
df2.write.format("com.databricks.spark.csv").option("header", "true").option("inferschema", "true")\
    .option("delimiter", ",").save("/content/drive/MyDrive/SoulCode/MiniProjeto/df_marketing_pySpark")